# Update Protein Identifiers to UniProt

Note: Requires internet connection to download information from the UniProt.

## Setup
### Import packages

In [1]:
from collections import defaultdict
import re
import pandas as pd
from rbc_gem_utils import (
    ROOT_PATH,
    DATABASE_PATH,
    EXTERNAL_PATH,
    ANNOTATION_PATH,
    INTERIM_PATH,
    check_version,
    check_database_version_online,
    show_versions,
    build_string,
)


from rbc_gem_utils.database.uniprot import (
    UNIPROT_ID_RE,
    UNIPROT_ISOFORM_ID_RE,
    UNIPROT_VERSION_EXPECTED,
    get_version_UniProt,
    query_UniProt,
    get_annotation_to_from_db_UniProt,
    get_label_miriam_mapping_UniProt,
    get_query_fields_UniProt,
    parse_isoforms_UniProt,
    parse_chains_UniProt,
    get_isoform_value_from_entry_UniProt,
)

# Display versions of last time notebook ran and worked
show_versions()


Package Information
-------------------
rbc-gem-utils 0.0.1

Dependency Information
----------------------
beautifulsoup4                       4.12.3
bio                                   1.6.2
cobra                                0.29.0
depinfo                               2.2.0
kaleido                               0.2.1
matplotlib                            3.8.2
memote                               0.17.0
networkx                              3.2.1
notebook                              7.0.7
openpyxl                              3.1.2
pandas                                2.2.0
pre-commit                            3.6.0
pyvis                                 0.3.2
rbc-gem-utils[database,network,vis] missing
requests                             2.31.0
scipy                                1.12.0
seaborn                              0.13.2

Build Tools Information
-----------------------
pip        23.3.1
setuptools 68.2.2
wheel      0.41.2

Platform Information
-------------------

## Check UniProt version
If the version does not match the expected version, it is because database has been updated since the last time this code was utilized. 

### Expected UniProt version: 2024_01
* According to [UniProt](https://www.uniprot.org/help/downloads), updates to the database are made every eight weeks. 
* Last release utilized: [2024_01](https://www.uniprot.org/uniprotkb/statistics) published on **Wed Jan 24 2024.**

In [2]:
if not check_database_version_online("UniProt"):
    warn(
        "Online version of database has been updated since the last time notebook was used."
    )

version = get_version_UniProt()
if check_version(version, UNIPROT_VERSION_EXPECTED, verbose=True):
    database_dirpath = f"{ROOT_PATH}{DATABASE_PATH}"
    annotation_dirpath = f"{ROOT_PATH}{ANNOTATION_PATH}"
else:
    # Use different directory paths for unexpected behavior
    database_dirpath = f"{ROOT_PATH}{INTERIM_PATH}"
    annotation_dirpath = f"{ROOT_PATH}{INTERIM_PATH}"

Current and expected versions match.


## Load aggregated proteomic data, raw

In [3]:
overwrite = True

excel_filepath_raw = (
    f"{ROOT_PATH}{EXTERNAL_PATH}/proteomics/proteomics_aggregated_raw.xlsx"
)

### Load obsolete identifiers

In [4]:
df_obsolete = pd.read_csv(
    f"{ROOT_PATH}{EXTERNAL_PATH}/proteomics/proteomics_obsolete.tsv",
    sep="\t",
    dtype=str,
    index_col=0,
)
for col in df_obsolete.columns:
    df_obsolete[col] = df_obsolete[col].str.split(";")
    df_obsolete = df_obsolete.explode(col)
df_obsolete

,UniProt,IPI,GI,UniParc,EntryNames
0,A0A024QZ64,NaN,78070601,NaN,NaN
1,A0A024R0L6,NaN,4505587,NaN,NaN
1,A0A024R0L6,NaN,14043830,NaN,NaN
2,A0A024R1A3,NaN,23510338,NaN,NaN
3,A0A024R1I3,NaN,10092677,NaN,NaN
...,...,...,...,...,...
2794,NaN,IPI00843857,NaN,NaN,NaN
2795,NaN,IPI00843921,NaN,NaN,NaN
2796,NaN,IPI00844038,NaN,NaN,NaN
2797,NaN,IPI00844133,NaN,NaN,NaN


### Load table of contents, raw

In [5]:
df_contents_raw = pd.read_excel(
    excel_filepath_raw, sheet_name="Table of Contents", dtype=str
).fillna("")
df_contents_raw

,PubMed/Sheet Name,ID type,Year,Publication,Notes
0,16861337,IPI,2006,"Pasini EM, Kirkegaard M, Mortensen P, Lutz HU,...",
1,18399644,IPI,2008,"Simó C, Bachi A, Cattaneo A, Guerrier L, Forti...",
2,18494517,IPI,2008,"Ringrose JH, van Solinge WW, Mohammed S, O'Fla...",
3,18614565,IPI,2008,"Roux-Dalvai F, Gonzalez de Peredo A, Simó C, G...",
4,19778645,GI,2009,"van Gestel RA, van Solinge WW, van der Toorn H...",
5,22954596,UniProt,2012,"Pesciotta EN, Sriswasdi S, Tang HY, Mason PJ, ...",
6,23781972,GI,2013,"Pallotta V, D'Alessandro A, Rinalducci S, Zoll...",
7,24555563,UniProt,2014,"Lange PF, Huesgen PF, Nguyen K, Overall CM. An...",
8,26078478,UniProt,2015,"Hegedűs T, Chaubey PM, Várady G, Szabó E, Sara...",
9,26474164,UniProt,2015,"Pesciotta EN, Lam HS, Kossenkov A, Ge J, Showe...",


### Map other identifiers to UniProt

In [6]:
updated_table_dict = {"Table of Contents": df_contents_raw.copy()}

index_name = "Uniprot"
for idx, (sheet_name, id_type) in df_contents_raw[
    ["PubMed/Sheet Name", "ID type"]
].iterrows():
    df = pd.read_excel(
        excel_filepath_raw, sheet_name=sheet_name, usecols=[0], dtype=str
    )
    df.columns = [id_type]
    if id_type != "UniProt":
        # Map to obsolete identifiers
        df_obsolete_mapping = (
            df_obsolete[[id_type, "UniProt"]].dropna(how="all").drop_duplicates().copy()
        )
        if id_type == "GI":
            df[id_type] = df[id_type].apply(
                lambda x: x.split("|")[-1] if str(x).startswith("gi|") else x
            )
        df_obsolete_mapping = df.merge(
            df_obsolete_mapping, left_on=id_type, right_on=id_type, how="left"
        )
        counts = df_obsolete_mapping.nunique()
        counts["Obsolete"] = (
            df_obsolete_mapping["UniProt"].isna().value_counts()[True].item()
        )
        updated_table_dict["Table of Contents"].loc[idx, "ID type"] = "UniProt"
        updated_table_dict["Table of Contents"].loc[idx, "Notes"] = build_string(
            [f"{v} {k}" for k, v in counts.to_dict().items()]
        )
        updated_table_dict[sheet_name] = list(
            df_obsolete_mapping["UniProt"].dropna().unique()
        )
    else:
        df["UniProt"] = df["UniProt"].apply(
            lambda x: UNIPROT_ID_RE.search(x).group() if UNIPROT_ID_RE.search(x) else x
        )
        updated_table_dict[sheet_name] = list(df["UniProt"].dropna().unique())
        updated_table_dict["Table of Contents"].loc[idx, "Notes"] = "{} UniProt".format(
            df["UniProt"].nunique()
        )

updated_table_dict["Table of Contents"]

,PubMed/Sheet Name,ID type,Year,Publication,Notes
0,16861337,UniProt,2006,"Pasini EM, Kirkegaard M, Mortensen P, Lutz HU,...",590 IPI;438 UniProt;147 Obsolete
1,18399644,UniProt,2008,"Simó C, Bachi A, Cattaneo A, Guerrier L, Forti...",153 IPI;135 UniProt;18 Obsolete
2,18494517,UniProt,2008,"Ringrose JH, van Solinge WW, Mohammed S, O'Fla...",699 IPI;400 UniProt;299 Obsolete
3,18614565,UniProt,2008,"Roux-Dalvai F, Gonzalez de Peredo A, Simó C, G...",1577 IPI;1318 UniProt;256 Obsolete
4,19778645,UniProt,2009,"van Gestel RA, van Solinge WW, van der Toorn H...",561 GI;473 UniProt;138 Obsolete
5,22954596,UniProt,2012,"Pesciotta EN, Sriswasdi S, Tang HY, Mason PJ, ...",831 UniProt
6,23781972,UniProt,2013,"Pallotta V, D'Alessandro A, Rinalducci S, Zoll...",140 GI;153 UniProt;27 Obsolete
7,24555563,UniProt,2014,"Lange PF, Huesgen PF, Nguyen K, Overall CM. An...",1220 UniProt
8,26078478,UniProt,2015,"Hegedűs T, Chaubey PM, Várady G, Szabó E, Sara...",419 UniProt
9,26474164,UniProt,2015,"Pesciotta EN, Lam HS, Kossenkov A, Ge J, Showe...",1245 UniProt


### Map UniProt IDs to current UniProt

In [7]:
# Extract all relevant information for now and save
query_parameters = {
    "query": " && ".join(
        [
            "(organism_id:9606)",  # Homo sapiens (Human)
        ]
    ),
    "format": "tsv",
    "size": 500,
    "compressed": True,
    "fields": ",".join(
        [
            "reviewed",
            "accession",
            "gene_primary",
        ]
    ),
}
idx = 0
for sheet_name, query_ids in updated_table_dict.copy().items():
    if sheet_name == "Table of Contents":
        continue
    print(f"{sheet_name}\n{len(sheet_name) * '-'}")
    df_results, uniparc, failed_ids, unmapped_ids = query_UniProt(
        query_ids,
        query_parameters=query_parameters,
        from_db="UniProtKB",
        to_db="UniProtKB",
        return_failed=True,
    )
    df_uniprot = df_results["Entry"].drop_duplicates().dropna().reset_index(drop=True)
    df_uniprot.name = index_name
    updated_table_dict[sheet_name] = df_uniprot
    updated_table_dict["Table of Contents"].loc[
        idx, "Notes"
    ] += f" -->  {len(df_uniprot)} UniProt {version}"
    idx += 1
    print()

updated_table_dict["Table of Contents"]

16861337
--------
Fetched: 441 / 441

18399644
--------
Fetched: 136 / 136

18494517
--------
Fetched: 400 / 400

18614565
--------
Fetched: 500 / 1321
Fetched: 1000 / 1321
Fetched: 1321 / 1321

19778645
--------
Fetched: 473 / 473

22954596
--------
Fetched: 500 / 765


Number of failed query IDs : 67


Fetched: 765 / 765


Number of failed IDs : 8
Number of obsolete IDs : 54



23781972
--------
Fetched: 153 / 153

24555563
--------
Fetched: 500 / 1214
Fetched: 1000 / 1214


...


33806028
--------
Fetched: 500 / 1343
Fetched: 1000 / 1343
Fetched: 1343 / 1343

35858567
--------
Fetched: 500 / 1530
Fetched: 1000 / 1530
Fetched: 1500 / 1530
Fetched: 1530 / 1530

36346805
--------
Fetched: 500 / 659
Fetched: 659 / 659

37760001
--------
Fetched: 500 / 878
Fetched: 878 / 878

37942280
--------
Fetched: 500 / 1813
Fetched: 1000 / 1813
Fetched: 1500 / 1813


Number of failed query IDs : 1


Fetched: 1813 / 1813


Number of obsolete IDs : 1


,PubMed/Sheet Name,ID type,Year,Publication,Notes
0,16861337,UniProt,2006,"Pasini EM, Kirkegaard M, Mortensen P, Lutz HU,...",590 IPI;438 UniProt;147 Obsolete --> 441 UniP...
1,18399644,UniProt,2008,"Simó C, Bachi A, Cattaneo A, Guerrier L, Forti...",153 IPI;135 UniProt;18 Obsolete --> 136 UniPr...
2,18494517,UniProt,2008,"Ringrose JH, van Solinge WW, Mohammed S, O'Fla...",699 IPI;400 UniProt;299 Obsolete --> 400 UniP...
3,18614565,UniProt,2008,"Roux-Dalvai F, Gonzalez de Peredo A, Simó C, G...",1577 IPI;1318 UniProt;256 Obsolete --> 1321 U...
4,19778645,UniProt,2009,"van Gestel RA, van Solinge WW, van der Toorn H...",561 GI;473 UniProt;138 Obsolete --> 473 UniPr...
5,22954596,UniProt,2012,"Pesciotta EN, Sriswasdi S, Tang HY, Mason PJ, ...",831 UniProt --> 765 UniProt 2024_01
6,23781972,UniProt,2013,"Pallotta V, D'Alessandro A, Rinalducci S, Zoll...",140 GI;153 UniProt;27 Obsolete --> 153 UniPro...
7,24555563,UniProt,2014,"Lange PF, Huesgen PF, Nguyen K, Overall CM. An...",1220 UniProt --> 1214 UniProt 2024_01
8,26078478,UniProt,2015,"Hegedűs T, Chaubey PM, Várady G, Szabó E, Sara...",419 UniProt --> 416 UniProt 2024_01
9,26474164,UniProt,2015,"Pesciotta EN, Lam HS, Kossenkov A, Ge J, Showe...",1245 UniProt --> 1148 UniProt 2024_01


### Export aggregated proteomic data, updated IDs


In [8]:
if overwrite:
    with pd.ExcelWriter(
        f"{ROOT_PATH}{EXTERNAL_PATH}/proteomics/proteomics_aggregated.xlsx"
    ) as writer:
        for sheet_name, df in updated_table_dict.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)
else:
    with pd.ExcelWriter(
        f"{ROOT_PATH}{INTERIM_PATH}/proteomics_aggregated.xlsx"
    ) as writer:
        for sheet_name, df in updated_table_dict.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)